# 資料索引(Indexing)與選取(Selection)

在NumPy陣列中訪問，設置和修改值的方法和工具包括索引（例如，``arr [2,1]``），切片（例如，``arr [：，1：5]``），遮罩（例如，``arr [arr> 0]`` ），花式索引（例如，``arr [0，[1,5]]``）及其組合（例如，``arr [：，[1,5]]``）。在Pandas``Series``和``DataFrame``物件中訪問和修改值有類似方法。

## Series中資料的選取

``Series``物件可用於一維NumPy陣列與標準的Python字典。記住這兩個重疊的類比，可幫助理解這些陣列中資料索引和選擇的模式。

## 將 Series當作dict

像字典一樣，``Series``j物件提供從一組鍵到一組值的映射：

In [1]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [2]:
data['b']

0.5

我們還可以使用類似字典的Python表達式和方法來檢查鍵/索引和值：

In [3]:
'a' in data

True

In [4]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

``Series``物件甚至可以用類似字典的語法修改。如同通過分配新鍵來擴展字典一樣，可以通過分配新的索引值來擴展``Series``：

In [6]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

物件的容易可變性是一個方便特性：在物件內，Pandas正在決定可能需要進行的記憶體佈局和數據複製; 用戶通常不需要擔心這些問題。

### 將Series當作一維陣列

一個``Series``建立在這個類似字典的界面上，並通過與NumPy陣列相同的基本機制提供陣列樣式的項目選擇 - 即* slices *，* masking *和* fancy indexing *。這些例子如下：

In [7]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [8]:
# slicing by implicit integer index
data[0:2]

a    0.25
b    0.50
dtype: float64

In [9]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [10]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

其中切片可能是混亂的根源。請留意，當使用顯式索引進行切片時（即``data ['a'：'c']``），最終索引在切片中包含*，而在使用隱式索引進行切片時（即`` data [0：2]``），最終索引從切片中排除*。

### 索引採用指令: loc, iloc, and ix

這些切片和索引約定可能會引起混淆。例如，如果你的``Series``有一個顯式的整數索引，那麼索引操作如``data [1]``將使用顯式索引，而切片操作如``data [1：3]`` 將使用隱式的Python風格索引。

In [11]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [12]:
# explicit index when indexing
data[1]

'a'

In [13]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

由於在整數索引的情況下存在這種潛在的混淆，Pandas提供了一些特殊的* indexer *屬性，這些屬性明確地顯示了某些索引方案。這些不是函數方法，而是將特定切片接口顯示給``Series``中的資料的屬性。

首先，``loc``屬性允許索引和切片始終引用顯式索引：

In [14]:
data.loc[1]

'a'

In [15]:
data.loc[1:3]

1    a
3    b
dtype: object

``iloc``屬性允許索引和切片引用隱式的Python樣式索引：

In [16]:
data.iloc[1]

'b'

In [17]:
data.iloc[1:3]

3    b
5    c
dtype: object

第三個索引屬性``ix``是兩者的混合，而``Series``物件相當於標準的``[]``索引。``ix``索引器的目的將在``DataFrame``物件的上下文中變得更加明顯。

Python代碼的一個指導原則是“顯式優於隱式”。``loc``和``iloc``的顯式特性使它們在維護清晰可讀的代碼時非常有用; 特別是在整數索引的情況下，儘量使用這兩者來使代碼更容易閱讀和理解，並防止由於混合索引/切片約定導致的細微錯誤。

## DataFrame中的資料選取 

``DataFrame``類同二維或結構化陣列與共享相同索引的``Series``結構字典。

### 將DataFrame當作字典(dictionary)

將``DataFrame``作為相關``Series``物件的字典。以美國各州面積與人口數的來舉例：

In [19]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 39250017,'Texas': 27862596,
                   'Florida': 20612439,'New York': 19745289,
                   'Illinois': 12801539})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,39250017
Florida,170312,20612439
Illinois,149995,12801539
New York,141297,19745289
Texas,695662,27862596


構成``DataFrame``行的單個``Series``可以通過column名稱的字典式索引來訪問：

In [20]:
data['area']

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

同樣，我們可以使用屬性樣式訪問，其column名是字串：

In [21]:
data.area

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

此屬性樣式行訪問實際上訪問與字典樣式訪問完全相同的物件：

In [22]:
data.area is data['area']

True

雖然這是一個有用的簡寫，但請記住，它並不適用於所有情況！
例如，如果行名不是字串，或者行名與“DataFrame”的方法衝突，則無法進行此屬性樣式的訪問。
例如，``DataFrame``有一個``pop（）``方法，所以``data.pop``將指向這個而不是``“pop”``行：

In [23]:
data.pop is data['pop']

False

特別是要避免通過屬性嘗試行分配的誘惑（即使用``data ['pop'] = z``而不是``data.pop = z``）。
與前面討論的``Series``物件一樣，這種字典式語法也可用於修改物件，在這種情況下添加一個新行：

In [24]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,39250017,92.578000
Florida,170312,20612439,121.027520
Illinois,149995,12801539,85.346438
New York,141297,19745289,139.743158
Texas,695662,27862596,40.051916


### DataFrame當作二維陣列

如前所述，我們還可以將``DataFrame``視為增強的二維陣列。我們可以使用``values``屬性檢查原始底層資料陣列：

In [25]:
data.values

array([[4.23967000e+05, 3.92500170e+07, 9.25780002e+01],
       [1.70312000e+05, 2.06124390e+07, 1.21027520e+02],
       [1.49995000e+05, 1.28015390e+07, 8.53464382e+01],
       [1.41297000e+05, 1.97452890e+07, 1.39743158e+02],
       [6.95662000e+05, 2.78625960e+07, 4.00519160e+01]])

考慮到這一點，許多熟悉的類似陣列的觀察可以在``DataFrame``本身上完成。例如，我們可以將完整的``DataFrame``轉置為交換行和列：

In [26]:
data.T

,California,Florida,Illinois,New York,Texas
area,4.239670e+05,1.703120e+05,1.499950e+05,1.412970e+05,6.956620e+05
pop,3.925002e+07,2.061244e+07,1.280154e+07,1.974529e+07,2.786260e+07
density,9.257800e+01,1.210275e+02,8.534644e+01,1.397432e+02,4.005192e+01


然而，當談到“DataFrame”物件的索引時，很明顯行的字典式索引排除了我們將其簡單地視為NumPy陣列的能力。
特別是，將單個索引傳遞給陣列會訪問一列：

In [27]:
data.values[0]

array([4.23967000e+05, 3.92500170e+07, 9.25780002e+01])

將單個“索引”傳遞給``DataFrame``可以訪問一行：

In [28]:
data['area']

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

因此，對於陣列樣式的索引，我們需要另一個約定。
Pandas再次使用前面提到的``loc``，``iloc``和``ix``索引器。使用``iloc``索引器，我們可以將底層陣列索引為好像它是一個簡單的NumPy陣列（使用隱式的Python樣式索引），但結果中保留了``DataFrame``索引和行標籤：

In [29]:
data.iloc[:3, :2]

,area,pop
California,423967,39250017
Florida,170312,20612439
Illinois,149995,12801539


類似地，使用``loc``索引器，我們可以使用顯式索引和行名稱以類似陣列的樣式索引基礎資料：

In [30]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,39250017
Florida,170312,20612439
Illinois,149995,12801539


``ix``索引器允許這兩種方法的混合：

In [31]:
data.ix[:3, :'pop']

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,area,pop
California,423967,39250017
Florida,170312,20612439
Illinois,149995,12801539


請記住，對於整數索引，``ix``索引器受到與整數索引的``Series``物件討論的相同的潛在混淆來源。

任何熟悉的NumPy風格的資料訪問模式都可以在這些索引器中使用。
例如，在``loc``索引器中，我們可以組合遮罩和花式索引，如下所示：

In [32]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
Florida,20612439,121.027520
New York,19745289,139.743158


任何這些索引約定也可用於設置或修改值; 也可以使用NumPy的標準方式完成：

In [33]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,39250017,90.000000
Florida,170312,20612439,121.027520
Illinois,149995,12801539,85.346438
New York,141297,19745289,139.743158
Texas,695662,27862596,40.051916


### 其他索引約定

有一些額外的索引約定可能與前面的討論不一致，但在實踐中可能非常有用。
首先，* indexing *引用行，* slicing *引用列：

In [34]:
data['Florida':'Illinois']

,area,pop,density
Florida,170312,20612439,121.027520
Illinois,149995,12801539,85.346438


這樣的切片也可以通過數字而不是索引來引用列：

In [35]:
data[1:3]

,area,pop,density
Florida,170312,20612439,121.027520
Illinois,149995,12801539,85.346438


類似地，直接遮罩操作也是按列而不是按行解釋的：

In [36]:
data[data.density > 100]

,area,pop,density
Florida,170312,20612439,121.027520
New York,141297,19745289,139.743158


這兩個約定在語法上類似於NumPy陣列上的約定，雖然這些約定可能不完全符合Pandas約定的模型，但它們在實踐中非常有用。